<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=90b8d0fc3f98103ae5dbbe642fb1b2e14fd4bdd4da1225f278a6d603844fd7cf
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
import folium
import wget
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`

---
- _TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map
- An example of folium.Circle:
- `folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`
- An example of folium.Marker:
- `folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`




In [78]:
# Initial the map
# Initialize the map centered at NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for NASA Johnson Space Center
folium.Circle(
    location=nasa_coordinate,
    radius=500,  # Radius in meters
    color='#d35400',  # Orange color
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)

folium.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>NASA JSC</b></div>',
    )
).add_to(site_map)

# Add a Circle and Marker for each launch site
for index, site in launch_sites_df.iterrows():
    # Extract the coordinates and name of the launch site
    coordinate = [site['Lat'], site['Long']]
    launch_site_name = site['Launch Site']

    # Add a Circle for each launch site
    folium.Circle(
        location=coordinate,
        radius=500,  # 1000 meters = 1 km
        color='#000000',  # Black circle
        fill=True
    ).add_child(folium.Popup(launch_site_name)).add_to(site_map)

    # Add a Marker for each launch site
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 20; color:#EA0000;"><b>{launch_site_name}</b></div>',
        )
    ).add_to(site_map)

# Display the map
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [17]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [18]:
marker_cluster = MarkerCluster().add_to(site_map)


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [20]:
# Apply a function to set marker color based on the class value
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Display the updated DataFrame
print(spacex_df)

     Launch Site        Lat        Long  class marker_color
0    CCAFS LC-40  28.562302  -80.577356      0          red
1    CCAFS LC-40  28.562302  -80.577356      0          red
2    CCAFS LC-40  28.562302  -80.577356      0          red
3    CCAFS LC-40  28.562302  -80.577356      0          red
4    CCAFS LC-40  28.562302  -80.577356      0          red
5    CCAFS LC-40  28.562302  -80.577356      0          red
6    CCAFS LC-40  28.562302  -80.577356      0          red
7    CCAFS LC-40  28.562302  -80.577356      0          red
8    CCAFS LC-40  28.562302  -80.577356      0          red
9    CCAFS LC-40  28.562302  -80.577356      0          red
10   CCAFS LC-40  28.562302  -80.577356      0          red
11   CCAFS LC-40  28.562302  -80.577356      0          red
12   CCAFS LC-40  28.562302  -80.577356      0          red
13   CCAFS LC-40  28.562302  -80.577356      0          red
14   CCAFS LC-40  28.562302  -80.577356      0          red
15   CCAFS LC-40  28.562302  -80.577356 

In [ ]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [22]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

# Iterate through each row in spacex_df to add markers
for index, record in spacex_df.iterrows():
    coordinate = [record['Lat'], record['Long']]
    launch_class = record['class']

    # Set marker color based on launch success or failure
    marker_color = 'green' if launch_class == 1 else 'red'

    # Create a Marker object
    marker = folium.Marker(
        location=coordinate,
        icon=folium.Icon(color=marker_color, icon_color='white'),
        popup=f"Launch Site: {record['Launch Site']}<br>Class: {'Success' if launch_class == 1 else 'Failure'}"
    )

    # Add the marker to the marker_cluster
    marker_cluster.add_child(marker)

# Add marker_cluster to the site_map
site_map.add_child(marker_cluster)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [23]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [36]:
launch_site_lat_1 = 34.63281
launch_site_lon_1 = -120.61061
coastline_lat_1 = 34.63479
coastline_long_1 = -120.62537
launch_site_lat_2 = 28.57298
launch_site_lon_2 = -80.6471
coastline_lat_2 = 28.57102
coastline_long_2 = -80.60676
launch_site_lat_3 = 28.56258
launch_site_lon_3 = -80.57723
coastline_lat_3 = 28.56386
coastline_long_3 = -80.56788

#  call the distance function
distance_coastline_1 = calculate_distance(launch_site_lat_1, launch_site_lon_1, coastline_lat_1, coastline_long_1)
distance_coastline_2 = calculate_distance(launch_site_lat_2, launch_site_lon_2, coastline_lat_2, coastline_long_2)
distance_coastline_3 = calculate_distance(launch_site_lat_3, launch_site_lon_3, coastline_lat_3, coastline_long_3)

# print the results of the distance functon
print("VAFB SLC-4E: " + str(distance_coastline_1) + " \nKSC LC-39A: " + str(distance_coastline_2) + " \nCAFS LC-40: " + str(distance_coastline_3))

VAFB SLC-4E: 1.3686712579653866 
KSC LC-39A: 3.946594919493234 
CAFS LC-40: 0.9244500442587902


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [79]:
launch_site_1 = (34.63281, -120.61061)
coastline_1 = (34.63479, -120.62537)

launch_site_2 = (28.57298, -80.6471)
coastline_2 = (28.57102, -80.60676)

launch_site_3 = (28.56258, -80.57723)
coastline_3 = (28.56386, -80.56788)

# Add markers for each coastline distance

circle_1 = folium.Circle(coastline_1, radius=10, color='#6600CC', fil=False).add_child(folium.Popup('1.37 KM, Lat: 34.63479, Long: -120.62537'))
circle_2 = folium.Circle(coastline_2, radius=10, color='#6600CC', fill=False).add_child(folium.Popup('3.95 KM, Lat: 28.57102, Long: -80.60676'))
circle_3 = folium.Circle(coastline_3, radius=10, color='#6600CC', fill=False).add_child(folium.Popup('.92 KM, Lat: 28.56386, Long:-80.56788'))

# Marker for VAFB SLC-4E
folium.Marker(
    coastline_1,
    icon=DivIcon(
        icon_size=(5, 5),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_1),
    )
).add_to(site_map)

# # Marker for KSC LC-39A
folium.Marker(
    coastline_2,
    icon=DivIcon(
        icon_size=(5, 5),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_2),
    )
).add_to(site_map)

# Marker for CCAFS LC-40
folium.Marker(
    coastline_3,
    icon=DivIcon(
        icon_size=(5, 5),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_3),
    )
).add_to(site_map)

site_map.add_child(circle_1)
site_map.add_child(circle_2)
site_map.add_child(circle_3)

# Display the map
site_map

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [80]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

VAFB = folium.PolyLine(locations=[launch_site_1,coastline_1], weight=3)
KSC = folium.PolyLine(locations=[launch_site_2,coastline_2], weight=3)
CCAFS = folium.PolyLine(locations=[launch_site_3,coastline_3], weight=3)
site_map.add_child(VAFB)
site_map.add_child(KSC)
site_map.add_child(CCAFS)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [92]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_lat_1 = 34.63281
launch_site_lon_1 = -120.61061
railway_lat_1 = 34.63969
railway_long_1 = -120.62207
city_lat_1 = 34.6384
city_long_1 = -120.45797
highway_lat_1 = 34.6384
highway_long_1 = -120.45797

#  call the distance function
distance_railway_1 = calculate_distance(launch_site_lat_1, launch_site_lon_1, railway_lat_1, railway_long_1)
distance_city_1 = calculate_distance(launch_site_lat_1, launch_site_lon_1, city_lat_1, city_long_1)
distance_highway_1 = calculate_distance(launch_site_lat_1, launch_site_lon_1, highway_lat_1, highway_long_1)

# print the results of the distance functon
print("Distance to railway: {:.2f}".format(distance_railway_1) + " KM" + " \nDistance to City: {:.2f}".format(distance_city_1) + " KM" + " \nDistance to highway: {:.2f}".format(distance_highway_1) + " KM")

Distance to railway: 1.30 KM 
Distance to City: 13.98 KM 
Distance to highway: 13.98 KM


In [93]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_lat_2 = 28.57298
launch_site_lon_2 = -80.6471
railway_lat_2 = 28.57969
railway_long_2 = -80.80513
city_lat_2 = 28.47618
city_long_2 = -80.77764
highway_lat_2 = 28.57389
highway_long_2 = -80.65569

#  call the distance function
distance_railway_2 = calculate_distance(launch_site_lat_2, launch_site_lon_2, railway_lat_2, railway_long_2)
distance_city_2 = calculate_distance(launch_site_lat_2, launch_site_lon_2, city_lat_2, city_long_2)
distance_highway_2 = calculate_distance(launch_site_lat_2, launch_site_lon_2, highway_lat_2, highway_long_2)

# print the results of the distance functon
# print the results of the distance functon
print("Distance to railway: {:.2f}".format(distance_railway_2) + " KM" + " \nDistance to City: {:.2f}".format(distance_city_2) + " KM" + " \nDistance to highway: {:.2f}".format(distance_highway_2) + " KM")

Distance to railway: 15.45 KM 
Distance to City: 16.69 KM 
Distance to highway: 0.85 KM


In [94]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_lat_3 = 28.56258
launch_site_lon_3 = -80.57723
railway_lat_3 = 28.57969
railway_long_3 = -80.80513
city_lat_3 = 28.39552
city_long_3 = -80.60561
highway_lat_3 = 28.49281
highway_long_3 = -80.5804

#  call the distance function
distance_railway_3 = calculate_distance(launch_site_lat_3, launch_site_lon_3, railway_lat_3, railway_long_3)
distance_city_3 = calculate_distance(launch_site_lat_3, launch_site_lon_3, city_lat_3, city_long_3)
distance_highway_3 = calculate_distance(launch_site_lat_3, launch_site_lon_3, highway_lat_3, highway_long_3)

# print the results of the distance functon
# print the results of the distance functon
print("Distance to railway: {:.2f}".format(distance_highway_3) + " KM" + " \nDistance to City: {:.2f}".format(distance_city_3) + " KM" + " \nDistance to highway: {:.2f}".format(distance_highway_3) + " KM")

Distance to railway: 7.77 KM 
Distance to City: 18.79 KM 
Distance to highway: 7.77 KM


In [106]:
launch_site_1 = (34.63281, -120.61061)
coastline_1 = (34.63479, -120.62537)
railway_1 = (34.63969,-120.62207)
city_1 = (34.6384,-120.45797)
highway_1 = (34.6384,-120.45797)

launch_site_2 = (28.57298, -80.6471)
coastline_2 = (28.57102, -80.60676)
railway_2 = (28.57969,-80.80513)
city_2 = (28.47618,-80.77764)
highway_2 = (28.57389,-80.65569)

launch_site_3 = (28.56258, -80.57723)
coastline_3 = (28.56386, -80.56788)
railway_3 = (28.57969,-80.80513)
city_3 = (28.39552,-80.60561)
highway_3 = (28.49281,-80.5804)

# Add markers for each coastline distance
circle_1 = folium.Circle(coastline_1, radius=150, color='#6600CC', fil=False).add_child(folium.Popup('Lat: 34.63479, Long: -120.62537'))
circle_2 = folium.Circle(coastline_2, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.57102, Long: -80.60676'))
circle_3 = folium.Circle(coastline_3, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.56386, Long:-80.56788'))
# circles for railway distance
circle_4 = folium.Circle(railway_1, radius=150, color='#6600CC', fil=False).add_child(folium.Popup('Lat: 34.63969, Long: -120.62207'))
circle_5 = folium.Circle(railway_2, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.57102, Long: -80.80513'))
circle_6 = folium.Circle(railway_3, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.57969, Long:-80.80513'))
# circles for city distance
circle_7 = folium.Circle(city_1, radius=150, color='#6600CC', fil=False).add_child(folium.Popup('Lat: 34.6384, Long: -120.45797'))
circle_8 = folium.Circle(city_2 , radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.47618, Long: -80.77764'))
circle_9 = folium.Circle(city_3, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.39552, Long:-80.60561'))
# circles for highway distance
circle_10 = folium.Circle(highway_1, radius=150, color='#6600CC', fil=False).add_child(folium.Popup('Lat: 34.6384, Long: -120.45797'))
circle_11 = folium.Circle(highway_2, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.57389, Long: -80.65569'))
circle_12 = folium.Circle(highway_3, radius=150, color='#6600CC', fill=False).add_child(folium.Popup('Lat: 28.49281, Long: -80.5804'))

# # Marker for VAFB SLC-4E
folium.Marker(
    coastline_1,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_1),
    )
).add_to(site_map)

# # Marker for VAFB SLC-4E
folium.Marker(
    railway_1,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_1),
    )
).add_to(site_map)

# # Marker for VAFB SLC-4E
folium.Marker(
    city_1,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 0; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_1),
    )
).add_to(site_map)

# # Marker for VAFB SLC-4E
folium.Marker(
    highway_1,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_1),
    )
).add_to(site_map)

# # Marker for KSC LC-39A
folium.Marker(
    coastline_2,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_2),
    )
).add_to(site_map)

# # Marker for KSC LC-39A
folium.Marker(
    railway_2,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_2),
    )
).add_to(site_map)

# # Marker for KSC LC-39A
folium.Marker(
    city_2,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_2),
    )
).add_to(site_map)

# # Marker for KSC LC-39A
folium.Marker(
    highway_2,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_2),
    )
).add_to(site_map)

# Marker for CCAFS LC-40
folium.Marker(
    coastline_3,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_3),
    )
).add_to(site_map)

# Marker for CCAFS LC-40
folium.Marker(
    railway_3,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_3),
    )
).add_to(site_map)

# Marker for CCAFS LC-40
folium.Marker(
    city_3,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_3),
    )
).add_to(site_map)

# Marker for CCAFS LC-40
folium.Marker(
    highway_3,
    icon=DivIcon(
        icon_size=(10, 10),
        icon_anchor=(0, 0),
        html='<div style="font-size: 20; color:#EA0000;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_3),
    )
).add_to(site_map)


site_map.add_child(circle_1)
site_map.add_child(circle_2)
site_map.add_child(circle_3)
site_map.add_child(circle_4)
site_map.add_child(circle_5)
site_map.add_child(circle_6)
site_map.add_child(circle_7)
site_map.add_child(circle_8)
site_map.add_child(circle_9)
site_map.add_child(circle_10)
site_map.add_child(circle_11)
site_map.add_child(circle_12)

VAFB_r = folium.PolyLine(locations=[launch_site_1,railway_1], weight=3)
VAFB_c = folium.PolyLine(locations=[launch_site_1,city_1], weight=3)
VAFB_h = folium.PolyLine(locations=[launch_site_1,highway_1], weight=3)

KSC_r = folium.PolyLine(locations=[launch_site_2,railway_2], weight=3)
KSC_c = folium.PolyLine(locations=[launch_site_2,city_2], weight=3)
KSC_h = folium.PolyLine(locations=[launch_site_2,highway_2], weight=3)

CCAFS_r = folium.PolyLine(locations=[launch_site_3,railway_3], weight=3)
CCAFS_c = folium.PolyLine(locations=[launch_site_3,city_3], weight=3)
CCAFS_h = folium.PolyLine(locations=[launch_site_3,highway_3], weight=3)

site_map.add_child(VAFB_r)
site_map.add_child(VAFB_c)
site_map.add_child(VAFB_h)

site_map.add_child(KSC_r)
site_map.add_child(KSC_c)
site_map.add_child(KSC_h)

site_map.add_child(CCAFS_r)
site_map.add_child(CCAFS_c)
site_map.add_child(CCAFS_h)

# Display the map
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
